# Neuronales Netz zur Realisierung einer DNF

## 1.Zuerst wird die DNF erstellt

In [91]:
import numpy as np

n = 10 #Anzahl der Eingabevariablen
m = 5 #Anzahl der Monome


#Definition der Monome als Liste von Literalen
# 1 für wahr, -1 für falsch, 0 für nicht verwendet
dnf = np.array([
    [ 1, -1,  1,  0,  0,  0,  0,  0,  0,  0],  # Monom 1: x1 ∧ ¬x2 ∧ x3
    [-1,  1,  0,  1,  0,  0,  0,  0,  0,  0],  # Monom 2: ¬x1 ∧ x2 ∧ x4
    [ 1,  0,  0, -1,  1,  0,  0,  0,  0,  0],  # Monom 3: x1 ∧ ¬x4 ∧ x5
    [ 0, -1,  1,  0,  1,  0,  0,  0,  0,  0],  # Monom 4: ¬x2 ∧ x3 ∧ x5
    [-1,  0,  0,  1, -1,  1,  0,  0,  0,  0],  # Monom 5: ¬x1 ∧ x4 ∧ ¬x5 ∧ x6
])

dnf2 = np.array([
    [ 1, -1,  1,  0,  0,  0,  0,  0,  0,  0],  # Monom 1: x1 ∧ ¬x2 ∧ x3
    [-1,  1,  -1,  0,  0,  0,  0,  0,  0,  0],  # Monom 2: ¬x1 ∧ x2 ∧ x4
    [ 1,  1,  -1,  0,  0,  0,  0,  0,  0,  0],  # Monom 3: x1 ∧ ¬x4 ∧ x5
    [ -1, -1,  1,  0,  0,  0,  0,  0,  0,  0],  # Monom 4: ¬x2 ∧ x3 ∧ x5
    [1,  -1,  -1,  0, 0,  0,  0,  0,  0,  0],  # Monom 5: ¬x1 ∧ x4 ∧ ¬x5 ∧ x6
])

## 2. Gewichtsmatrix und Schwellenwerte berechnen

In [92]:
def calculate_weights_and_thresholds(dnf_matrix):
    w = dnf_matrix.copy() #Gewichte für erste Schicht
    v = np.sum(np.abs(w), axis=1) #Schwellenwerte für jede Zeile
    W = np.ones((1, m))
    V = -(m - 1)
    print(f"w ={w}, v = {v},/n W = {W}/n V = {V}")
    return w, v, W, V

## 3.Vorzeichenfunktion definieren

In [93]:
def sgn(x):
    return np.where(x >= 0, 1, -1)

## 4.Vorwärtspropagation

In [94]:
def forward(x, w, v, W, V):
    z = sgn(np.dot(w, x) - v)
    print(z)
    y = sgn(np.dot(W, z) - V)
    print(y)
    return y, z

In [95]:
def train_step(x, p, w, v, W, V, eta):
    y, z = forward(x, w, v, W, V)
    error = p - y

    # ∆W1j = η · (p−y) · zj
    delta_W = eta * error * z
    # ∆V = −η · (p−y)
    delta_V = -eta * error

    # ∆wjk = η ·W1j(p−y) · xk
    delta_w = eta * error * W.T @ x.reshape(1, -1)
    # ∆vj = −η · (p−y) ·W1j
    delta_v = -eta * error * W.flatten()

    # Update
    w_new = w + delta_w
    v_new = v + delta_v
    W_new = W + delta_W.reshape(1, -1)
    V_new = V + delta_V

    return w_new, v_new, W_new, V_new, y, error

## 5. Tests mit Beispiel

In [96]:
# Beispielinput, der falsch sein soll (DNF = -1)
x = np.array([1, -1, -1, -1, 1, -1, -1, 1, 1, 1])
p = 1  # erwartete Zielausgabe für x

# Gewichte direkt aus DNF bestimmen
w, v, W, V = calculate_weights_and_thresholds(dnf2)

y, z = forward(x, w, v, W, V)


print(f"Output = {y}")
# Lernrate
eta = 0.1

# Gewichte und Schwellen vor dem Training speichern
#w_old, v_old, W_old, V_old = w.copy(), v.copy(), W.copy(), V



# Trainingsschritt
#w_new, v_new, W_new, V_new, y_pred, error = train_step(x, p, w, v, W, V, eta)

# ==========================
# 7. Ausgabe
# ==========================
print("Input x:           ", x)
#print("Erwarteter Output p:", p)
print("Netzoutput y:       ", y_pred)
print("Fehler (p - y):     ", error)

# Vergleich der Gewichte
print("\nHaben sich die Gewichte geändert?")
print("w verändert?:", not np.allclose(w_old, w_new))
print("v verändert?:", not np.allclose(v_old, v_new))
print("W verändert?:", not np.allclose(W_old, W_new))
print("V verändert?:", V_old != V_new)

w =[[ 1 -1  1  0  0  0  0  0  0  0]
 [-1  1 -1  0  0  0  0  0  0  0]
 [ 1  1 -1  0  0  0  0  0  0  0]
 [-1 -1  1  0  0  0  0  0  0  0]
 [ 1 -1 -1  0  0  0  0  0  0  0]], v = [3 3 3 3 3],/n W = [[1. 1. 1. 1. 1.]]/n V = -4
[-1 -1 -1 -1  1]
[1]
Output = [1]
Input x:            [ 1 -1 -1 -1  1 -1 -1  1  1  1]
Netzoutput y:        [-1]
Fehler (p - y):      [0]

Haben sich die Gewichte geändert?
w verändert?: False
v verändert?: False
W verändert?: False
V verändert?: [False]
